In [1]:
# Configuración del entorno de trabajo
# Importación de los datos
import sqlite3
# Para importar las librerías de 'pandas' y 'numpy' es necesario ejecutar: "pip install pandas numpy matplotlib seaborn scikit-learn jupyter"
import pandas as pd
import numpy as np
####################
import os
os.chdir("C:/Users/a21gebremedingl/Desktop/AD/Proyecto1_AD_Python")

## 1. CARGAR PEDIDOS (CSV)

In [2]:
# Cargar el fichero pedidos.csv
df_pedidos = pd.read_csv("sources/pedidos.csv", encoding="utf-16", sep="|") # Usando la codificación correcta: "utf-16"
print(df_pedidos)

      Id Pedido  id_cliente  id_producto  fecha_pedido  cantidad   \
0            1.0      1263.0        531.0    2025-02-28       7.0   
1            NaN         NaN        744.0    2023-11-18       8.0   
2            3.0      3060.0        990.0    2023-10-18      10.0   
3            4.0      1216.0        174.0    2024-05-31       3.0   
4            5.0      1558.0        916.0    2024-06-30       9.0   
...          ...         ...          ...           ...       ...   
39995    39996.0      4739.0        503.0    2025-07-15       3.0   
39996    39997.0      1020.0        351.0    2025-03-07       4.0   
39997    39998.0      4163.0        196.0    2025-01-26       6.0   
39998    39999.0      2882.0        521.0    2023-11-04       2.0   
39999    40000.0      2417.0        284.0            20       6.0   

      precio_unitario     total    metodo_pago  estado_pedido   \
0               356.14   2492.98        Tarjeta      Pendiente   
1              1569.27  12554.16  Trans

## 2. CARGAR CLIENTES (JSON)


In [3]:
# Cargar el fichero clientes.json
df_clientes = pd.read_json("sources/clientes.json")
print(df_clientes)

      id_cliente                nombre                        email  \
0              1           Leah Oliver       morrisgary@example.org   
1              2             Gary Yang       patrickorr@example.org   
2              3           Diane Black  shannonwilliams@example.org   
3              4       Stephanie Ochoa     thorntonmary@example.net   
4              5       Manuel Petersen           judy84@example.org   
...          ...                   ...                          ...   
4995        4996          Nathan Smith     petersonpaul@example.com   
4996        4997            Wendy Owen       susanrusso@example.net   
4997        4998  Christopher Thompson       nicholas14@example.com   
4998        4999          Cindy Rogers      whitejoshua@example.com   
4999        5000          Selena Mills       wolfsteven@example.org   

                    telefono  \
0      001-330-515-0970x7851   
1               971.920.5709   
2        (431)706-3322x22301   
3     001-651-540-3

## 3. CARGAR PRODUCTOS (SQLite)


In [4]:
# Cargar el fichero productos.db
conexion = sqlite3.connect("sources/productos.db")
with conexion:
    df_productos = pd.read_sql_query("SELECT * FROM productos", conexion)   #  Acceso desde pandas: Convierte la tabla SQL en DataFrame directamente
print(df_productos)

#conexion.close() -->  Cerrar la conexión a la base de datos. // En este caso como se usa 'with', se cierra automáticamente al salir del bloque

    id_producto nombre_producto    categoria precio_base descuento   stock  \
0           1.0          Home X       Libros      469.99       0.0   367.0   
1          None      Threat Pro      Deporte      644.21      20.0    98.0   
2             3        Reduce X         Ropa     1417.93      15.0   309.0   
3           4.0        Half Max      Deporte      579.69      20.0   442.0   
4           5.0     Control Pro      Deporte      373.93       0.0  -99999   
..          ...             ...          ...         ...       ...     ...   
995       996.0          Lose X       Libros      782.92      10.0    47.0   
996       997.0      Degree Pro  Electrónica     1810.45      15.0   469.0   
997       998.0   Describe Lite           Ho     1735.67       0.0   360.0   
998       999.0        Hope Max        Hogar        8.16      15.0   277.0   
999      1000.0   Standard Lite  Electrónica      800.65      20.0   137.0   

                           proveedor  fecha_alta  
0           

## 4. NORMALIZAR TIPOS DE CLAVE

In [5]:
# Convertimos los valores de las columnas que actuarán de clave a str para evitar conflictos
df_clientes['id_cliente'] = df_clientes['id_cliente'].astype(str)
df_productos['id_producto'] = df_productos['id_producto'].astype(str)

# Como la intencion es hacer un merge en 'df_pedidos' utilizando 'id_cliente' e 'id_producto', es importante asegurarse de que los nombres de las columnas coincidan exactamente
df_pedidos.columns = df_pedidos.columns.str.strip() # Eliminar espacios antes y después de cada nombre de columna de df_pedidos
df_pedidos['id_producto'] = df_pedidos['id_producto'].astype(str)
df_pedidos['id_cliente'] = df_pedidos['id_cliente'].astype(str)

## 5. UNIÓN DE DATASETS

In [6]:
# Unir los tres datasets en uno solo
df_global = df_pedidos.merge(df_clientes, on='id_cliente', how='left')
df_global = df_pedidos.merge(df_productos, on='id_producto', how='left')
print(df_global)

      Id Pedido id_cliente id_producto fecha_pedido cantidad precio_unitario  \
0           1.0     1263.0       531.0   2025-02-28      7.0          356.14   
1           NaN        nan       744.0   2023-11-18      8.0         1569.27   
2           3.0     3060.0       990.0   2023-10-18     10.0          707.74   
3           4.0     1216.0       174.0   2024-05-31      3.0         1281.67   
4           5.0     1558.0       916.0   2024-06-30      9.0         1208.76   
...         ...        ...         ...          ...      ...             ...   
48658   39996.0     4739.0       503.0   2025-07-15      3.0          543.26   
48659   39997.0     1020.0       351.0   2025-03-07      4.0          134.56   
48660   39998.0     4163.0       196.0   2025-01-26      6.0         1855.47   
48661   39999.0     2882.0       521.0   2023-11-04      2.0          939.56   
48662   40000.0     2417.0       284.0           20      6.0           594.5   

          total    metodo_pago estado_p

## Apartado 1: Información básica del dataset


In [7]:
# 1.1 Mostrar las primeras 5 filas del dataset global
print(f"\n== Mostrando las 5 primeras filas del dataset global ==\n{df_global.head(5)}")

# 1.2 Mostrar las dimensiones del dataset (filas y columnas)
print(f"\n== Mostrando las dimensiones del dataset (filas y columnas) ==\n{df_global.shape}")

# 1.3 Mostrar los nombres de todas las columnas
print(f"\n== Mostrando los nombres de todas las columnas ==\n{df_global.columns.tolist()}")

# 1.4 Mostrar información sobre los tipos de datos de las columnas
''' 
Proceso de optimización y verificación de los tipos de datos (para mejorar el rendimiento) que tuve en un principio pero que luego modifiqué:

# Columnas que deben ser enteros
cols_enteros = ["Id Pedido", "id_cliente", "id_producto","cantidad", "stock"]
for col in cols_enteros:
    # Reemplazar valores inválidos por 0
    df_global[col] = df_global[col].replace(["###ERROR###", "-", None], "0")
    # Convertir a float primero (asegura que todos los valores sean numéricos)
    df_global[col] = df_global[col].astype(float)
    # Reemplazar valores NaN por 0
    df_global[col] = df_global[col].fillna(0)
    # Convertir a entero
    df_global[col] = df_global[col].astype(int)

# Columnas que deben ser float
cols_float = ["precio_unitario", "precio_base", "total", "descuento"]
for col in cols_float:
    df_global[col] = df_global[col].replace(["###ERROR###", "-", None], "0")
    df_global[col] = df_global[col].astype(float)
    df_global[col] = df_global[col].fillna(0)
'''
# Columnas que deben ser enteros
cols_enteros = ["Id Pedido", "id_cliente", "id_producto","cantidad", "stock"]
for col in cols_enteros:
    # Convertir todo a string y eliminar cualquier caracter que no sea dígito
    df_global[col] = df_global[col].astype(str).str.replace(r'[^0-9]', '', regex=True)
    # Convertir a numérico y reemplazar errores por 0
    df_global[col] = pd.to_numeric(df_global[col], errors='coerce').fillna(0)
    # Convertir a entero
    df_global[col] = df_global[col].astype(int)

# Columnas que deben ser float
cols_float = ["precio_unitario", "precio_base", "total", "descuento"]
for col in cols_float:
    # Convertir todo a string y eliminar caracteres que no sean dígitos o punto decimal
    df_global[col] = df_global[col].astype(str).str.replace(r'[^0-9.]', '', regex=True)
    # Convertir a numérico y reemplazar errores por 0.0
    df_global[col] = pd.to_numeric(df_global[col], errors='coerce').fillna(0.0)

# Salida de tipos de datos
print(f"\n== Mostrando información sobre los tipos de datos de las columnas ==\n{df_global.dtypes}")


# 1.5 Mostrar las últimas 3 filas del dataset
print(f"\n== Mostrando las 3 últimas filas del dataset global ==\n{df_global.tail(3)}")



== Mostrando las 5 primeras filas del dataset global ==
  Id Pedido id_cliente id_producto fecha_pedido cantidad precio_unitario  \
0       1.0     1263.0       531.0   2025-02-28      7.0          356.14   
1       NaN        nan       744.0   2023-11-18      8.0         1569.27   
2       3.0     3060.0       990.0   2023-10-18     10.0          707.74   
3       4.0     1216.0       174.0   2024-05-31      3.0         1281.67   
4       5.0     1558.0       916.0   2024-06-30      9.0         1208.76   

      total    metodo_pago estado_pedido        País envío nombre_producto  \
0   2492.98        Tarjeta     Pendiente           Namibia    Business Pro   
1  12554.16  Transferencia     Pendiente  Christmas Island       Serve Max   
2    7077.4         PayPal     Pendiente          Tanzania     Economy Max   
3   3845.01   Criptomoneda     Pendiente          Bulgaria        Only Max   
4  10878.84         PayPal       Enviado           Romania           Own X   

     categoria pr

## Apartado 2: Tipos de datos y valores nulos


In [8]:
# 2.1 Mostrar información detallada del DataFrame (tipos, memoria, nulos)
print(f"\n== Mostrando información detallada del DataFrame (tipos, memoria, nulos) == \n{df_global.info}")

# 2.2 Contar valores nulos por columna
print(f"\n== Mostrando cantidad de valores nulos por columna:\n{df_global.isnull().sum()}")

# 2.3 Mostrar solo las columnas que tienen valores nulos
print(f"\n== Mostrando solo las columnas que tienen valores nulos ==\n{df_global.columns[df_global.isnull().any()]}")

# 2.4 Calcular el porcentaje de valores nulos por columna
porcentaje_nulos = df_global.isnull().mean() * 100
print(f"\n== Mostrando el porcentaje de valores nulos por columna ==\n{porcentaje_nulos.round(2)}")

# 2.5 Verificar si hay filas completamente vacías
filas_vacias = df_global.isin([None, 0, 'NaN', 'nan', '###ERROR###']).all(axis=1)   # Según cómo se planteó más arriba, el 0 se considera vacío
if filas_vacias.any():
    print(f"\n== Mostrando filas completamente vacías ==\n{filas_vacias}")
else:
    print('\n== No se pueden mostrar las filas compltamente vacías porque no existe ninguna ==')



== Mostrando información detallada del DataFrame (tipos, memoria, nulos) == 
<bound method DataFrame.info of        Id Pedido  id_cliente  id_producto fecha_pedido  cantidad  \
0             10       12630         5310   2025-02-28        70   
1              0           0         7440   2023-11-18        80   
2             30       30600         9900   2023-10-18       100   
3             40       12160         1740   2024-05-31        30   
4             50       15580         9160   2024-06-30        90   
...          ...         ...          ...          ...       ...   
48658     399960       47390         5030   2025-07-15        30   
48659     399970       10200         3510   2025-03-07        40   
48660     399980       41630         1960   2025-01-26        60   
48661     399990       28820         5210   2023-11-04        20   
48662     400000       24170         2840           20        60   

       precio_unitario     total    metodo_pago estado_pedido  \
0       

## Apartado 3: Estadísticas descriptivas


In [9]:
# 3.1 Estadísticas descriptivas de todas las columnas numéricas
# Como se modificó el tipo de datos de las columnas numéricas, ahora se pueden calcular las estadísticas descriptivas
print(f"\n== Estadísticas descriptivas de todas las columnas numéricas ==\n{df_global.describe()}")

# 3.2 Estadísticas descriptivas de todas las columnas (numéricas y categóricas)
print(f"\n== Estadísticas descriptivas de todas las columnas (numéricas y categóricas) ==\n{df_global.describe(include='all')}")

# 3.3 Calcular la media, mediana y moda de la columna 'total'
media = np.mean(df_global['total'])
mediana = np.median(df_global['total'])
# Con pandas sería --> media = df_global['total'].mean(); mediana = df_global['total'].median()
moda = df_global['total'].mode()[0]
print(f"\n== Mostrando operaciones con la columna 'total' ==\n·Media: {media}\n·Mediana: {mediana}\n·Moda: {moda}")

# 3.4 Calcular el rango (máximo - mínimo) de la columna 'cantidad'
rango = np.ptp(df_global['cantidad'])
print(f"\n== Mostrando rango de la columna 'cantidad' ==\n{rango}")
# Con pandas sería --> rango = df_global['cantidad'].max() - df_global['cantidad'].min()

# 3.5 Calcular la desviación estándar y la varianza de la columna 'total'
desv_estandar = np.std(df_global['total'])
varianza = np.var(df_global['total'])
print(f"\n== Desviación estándar y varianza de la columna 'total' ==\n·Desviación estándar: {desv_estandar.round(3)}\n·Varianza: {varianza.round(3)}")

# 3.6 Contar valores únicos en columnas categóricas
valores_unicos = df_global.select_dtypes(include="object").nunique()
print(f"\n== Contando valores únicos en columnas categóricas ==\n{valores_unicos}")



== Estadísticas descriptivas de todas las columnas numéricas ==
           Id Pedido     id_cliente   id_producto      cantidad  \
count   48663.000000   48663.000000  48663.000000  48663.000000   
mean   191973.627561   25149.345211  21556.560076   1749.798636   
std    118664.063604   18606.543271  36429.931182  12910.488796   
min         0.000000       0.000000      0.000000      0.000000   
25%     92125.000000   11290.000000   2620.000000     30.000000   
50%    188020.000000   24010.000000   5850.000000     50.000000   
75%    294735.000000   37480.000000   9070.000000     80.000000   
max    999990.000000  999990.000000  99999.000000  99999.000000   

       precio_unitario         total   precio_base     descuento         stock  
count     48663.000000  48663.000000  48663.000000  48663.000000  48663.000000  
mean       2629.548913   6868.110733   2232.943656   1534.717629   3605.588722  
std       12729.681949  12802.371479  11213.761516  12252.431451  11505.079585  
min    

## Apartado 4: Filtrado simple

In [10]:
# 4.1 Filtrar pedidos con cantidad mayor a 5 unidades
pedidos_mas_5_unidades = df_global[df_global['cantidad'] > 5]
print(f"\n== Mostrando pedidos con 'cantidad' mayor a 5 unidades ==\n{pedidos_mas_5_unidades}")

# 4.2 Filtrar pedidos con total mayor a 1000 euros
pedidos_mas_100_euros = df_global[df_global['total'] > 1000]
print(f"\n== Mostrando pedidos con 'total' mayor a 1000 euros ==\n{pedidos_mas_100_euros}")

# 4.3 Filtrar pedidos con precio unitario menor a 50 euros
pedidos_menos_50_euros = df_global[df_global['precio_unitario'] < 50]
print(f"\n== Mostrando pedidos con 'precio_unitario' menor a 50 euros ==\n{pedidos_menos_50_euros}")

# 4.4 Filtrar pedidos con cantidad igual a 1
pedidos_cantidad_igual_1 = df_global[df_global['cantidad'] == 1]
print(f"\n== Mostrando pedidos con 'cantidad' igual a 1 ==\n{pedidos_cantidad_igual_1}")

# 4.5 Mostrar los primeros 5 pedidos grandes (cantidad > 5)
print(f"\n== Mostrando los 5 primeros pedidos con 'cantidad' mayor a 5 unidades ==\n{pedidos_mas_5_unidades.head(5)}")  # Como ya está definida antes, cojo la variable  'pedidos_mas_5_unidades'

# 4.6 Mostrar los primeros 5 pedidos caros (total > 1000€)
print(f"\n== Mostrando los 5 primeros pedidos con 'total' mayor a 1000 euros ==\n{pedidos_mas_100_euros.head(5)}")  # Como ya está definida antes, cojo la variable  'pedidos_mas_100_euros'



== Mostrando pedidos con 'cantidad' mayor a 5 unidades ==
       Id Pedido  id_cliente  id_producto fecha_pedido  cantidad  \
0             10       12630         5310   2025-02-28        70   
1              0           0         7440   2023-11-18        80   
2             30       30600         9900   2023-10-18       100   
3             40       12160         1740   2024-05-31        30   
4             50       15580         9160   2024-06-30        90   
...          ...         ...          ...          ...       ...   
48658     399960       47390         5030   2025-07-15        30   
48659     399970       10200         3510   2025-03-07        40   
48660     399980       41630         1960   2025-01-26        60   
48661     399990       28820         5210   2023-11-04        20   
48662     400000       24170         2840           20        60   

       precio_unitario     total    metodo_pago estado_pedido  \
0               356.14   2492.98        Tarjeta     Pendien

## Apartado 5: Filtrado con múltiples condiciones

In [11]:
# 5.1 Filtrar pedidos enviados con cantidad mayor a 3
pedidos_enviados_mayor_3 = df_global[(df_global['estado_pedido'] == "Enviado") & (df_global['cantidad'] > 3)]
print(f"\n== Mostrando los 5 primeros pedidos con 'cantidad' mayor a 5 unidades ==\n{pedidos_enviados_mayor_3}")

# 5.2 Filtrar pedidos de tarjeta o PayPal con total mayor a 500 euros
pedidos_tarjeta = df_global['metodo_pago'] == 'Tarjeta'
pedidos_paypal = df_global['metodo_pago'] == 'PayPal'
pedidos_total_mayor_500 = df_global['total'] > 500
print(f"\n== Mostrando los pedidos de tarjeta o PayPal con 'total' mayor a 500 euros ==\n{df_global[(pedidos_tarjeta | pedidos_paypal) & pedidos_total_mayor_500]}")

# 5.3 Filtrar pedidos pendientes o cancelados con cantidad menor a 2
pedidos_pendientes = df_global['estado_pedido'] == 'Pendiente'
pedidos_cancelados = df_global['estado_pedido'] == 'Cancelado'
pedidos_cantidad_menor_2 = df_global['cantidad'] < 2
print(f"\n== Mostrando los pedidos pendientes o cancelados con 'cantidad' menor a 2 ==\n{df_global[(pedidos_pendientes | pedidos_cancelados) & pedidos_cantidad_menor_2]}")

# 5.4 Filtrar pedidos enviados o devueltos con precio unitario mayor a 100
pedidos_enviado = df_global['estado_pedido'] == 'Enviado'
pedidos_devuelto = df_global['estado_pedido'] == 'Devuelto'
pedidos_precio_mayor_2 = df_global['precio_unitario'] > 100
print(f"\n== Mostrando los pedidos enviados o devueltos con 'precio_unitario' mayor a 100 ==\n{df_global[(pedidos_enviado | pedidos_devuelto) & pedidos_precio_mayor_2]}")

# 5.5 Mostrar los primeros 5 pedidos enviados grandes
# Como sabemos de antes que pedidos grandes --> cantidad > 5
pedidos_enviados_mas_5_unidades = df_global[(df_global['estado_pedido'] == 'Enviado') & (df_global['cantidad'] > 5)]
print(f"\n== Mostrando los primeros 5 pedidos enviados grandes ('cantidad' > 5) ==\n{pedidos_enviados_mas_5_unidades.head(5)}")

# 5.6 Mostrar los primeros 5 pedidos premium (método de pago Tarjeta o Paypal y que además su total sea mayor que 500)
# Como esas variables ya están definidas en esta celda, las voy a utilizar aquí
pedidos_premium = df_global[(pedidos_tarjeta | pedidos_paypal) & pedidos_total_mayor_500]
print(f"\n== Mostrando los primeros 5 pedidos premium (tarjeta o PayPal con 'total' mayor a 500 euros) ==\n{pedidos_premium.head(5)}")



== Mostrando los 5 primeros pedidos con 'cantidad' mayor a 5 unidades ==
       Id Pedido  id_cliente  id_producto fecha_pedido  cantidad  \
4             50       15580         9160   2024-06-30        90   
8             90        3740         5240   2024-09-25        30   
19           200       39230         8860   2024-03-19        40   
20           210       22250         4070   2024-04-29        80   
21           220       39780         6860          NaN        60   
...          ...         ...          ...          ...       ...   
48640     399780       21810         7870   2023-11-21        70   
48641     399790        6150         2050   2024-03-26        10   
48649     399870       24010         7930   2023-11-20        10   
48651     399890        7680         7250   2025-05-17        70   
48659     399970       10200         3510   2025-03-07        40   

       precio_unitario     total    metodo_pago estado_pedido   País envío  \
4              1208.76  10878.8

## Apartado 6: Filtrado temporal

In [12]:

# 6.1 Convertir la columna fecha_pedido a tipo datetime
print(df_global['fecha_pedido'])
# Como existen fechas con un formato erróneo, uso 'errors="coerce"' para indicar que no son una fecha válida asignando un valor nulo --> 'NaT'
df_global['fecha_pedido'] = pd.to_datetime(df_global['fecha_pedido'], errors="coerce")
print(f"\n== Mostrando las fechas de los pedidos con su nuevo formato (datetime) ==\n{df_global['fecha_pedido']}") 

# 6.2 Filtrar pedidos del año 2024
pedidos_2024 = df_global[df_global['fecha_pedido'].dt.year == 2024]     # Como la columna ya está convertida a 'datetime', se puede acceder directamente al año 
print(f"\n== Mostrando los pedidos del año 2024 ==\n{pedidos_2024}") 

# 6.3 Filtrar pedidos del mes de enero de 2024
pedidos_enero_2024 = df_global[(df_global['fecha_pedido'].dt.year == 2024) & (df_global['fecha_pedido'].dt.month == 1)]     # Pasa lo mismo que en el anterior, se puede acceder directamente al mes 
print(f"\n== Mostrando los pedidos de enero del año 2024 ==\n{pedidos_enero_2024}") 

# 6.4 Filtrar pedidos del primer trimestre de 2024
pedidos_primer_trimestre_2024 = df_global[(df_global['fecha_pedido'].dt.year == 2024) & (df_global['fecha_pedido'].dt.quarter == 1)]    # Es como se expresan los trimestres con 'datetime'
print(f"\n== Mostrando los pedidos del primer trimestre del año 2024 ==\n{pedidos_primer_trimestre_2024}") 

# 6.5 Filtrar pedidos de los lunes (día de la semana = 0)
pedidos_lunes = df_global[(df_global['fecha_pedido'].dt.weekday == 0)]
print(f"\n== Mostrando los pedidos de los lunes ==\n{pedidos_lunes}") 

# 6.6 Mostrar los primeros 5 pedidos de 2024
pedidos_2024 = df_global[df_global['fecha_pedido'].dt.year == 2024].head(5)
print(f"\n== Mostrando los 5 primeros pedidos del año 2024 ==\n{pedidos_2024}") 


0        2025-02-28
1        2023-11-18
2        2023-10-18
3        2024-05-31
4        2024-06-30
            ...    
48658    2025-07-15
48659    2025-03-07
48660    2025-01-26
48661    2023-11-04
48662            20
Name: fecha_pedido, Length: 48663, dtype: object

== Mostrando las fechas de los pedidos con su nuevo formato (datetime) ==
0       2025-02-28
1       2023-11-18
2       2023-10-18
3       2024-05-31
4       2024-06-30
           ...    
48658   2025-07-15
48659   2025-03-07
48660   2025-01-26
48661   2023-11-04
48662          NaT
Name: fecha_pedido, Length: 48663, dtype: datetime64[ns]

== Mostrando los pedidos del año 2024 ==
       Id Pedido  id_cliente  id_producto fecha_pedido  cantidad  \
3             40       12160         1740   2024-05-31        30   
4             50       15580         9160   2024-06-30        90   
7             80        2850         4810   2024-12-17        70   
8             90        3740         5240   2024-09-25        30   
10      

## Apartado 7: Estado del pedido

In [32]:
# 7.1 Contar el número de pedidos por cada estado
num_pedidos_estado = df_global.groupby('estado_pedido').size()
print(f"\n== Mostrando el número de pedidos por cada estado ==\n{num_pedidos_estado}") 

# 7.2 Calcular el porcentaje de pedidos por estado
# Echo mano de la variable asignada al dataframe con el número de pedidos por cada estado para llevar a cabo sus porcentajes
porcentajes_estados = ((num_pedidos_estado / num_pedidos_estado.sum()) * 100).round(2)
print(f"\n== Mostrando el porcentaje de pedidos por cada estado ==\n{porcentajes_estados}") 

# 7.3 Calcular el total de ventas por estado
total_ventas_estado = df_global.groupby('estado_pedido')['total'].sum()
print(f"\n== Mostrando el total de ventas por estado ==\n{total_ventas_estado}")    # Como se convirtió la columna 'total' a float, el resultado son datos totales numéricos

# 7.4 Calcular el valor promedio de pedidos por estado
valor_promedio_estado = (df_global.groupby('estado_pedido')['total'].mean()).round(2)
print(f"\n== Mostrando el valor promedio de pedidos por estado ==\n{valor_promedio_estado}") 

# 7.5 Calcular la cantidad total de productos vendidos por estado
cantidad_total_estado = df_global.groupby('estado_pedido')['cantidad'].sum()
print(f"\n== Mostrando la cantidad total de productos vendidos por estado ==\n{cantidad_total_estado}") 

# 7.6 Mostrar estadísticas completas por estado (numero de pedidos, total de ventas, promedio de pedidos, cantidad total, promedio de la cantidad)
estadisticas_estado = df_global.groupby("metodo_pago").agg(
    numero_pedidos = ("Id Pedido", "count"),
    total_ventas = ("total", "sum"),
    promedio_ventas = ("total", "mean"),
    cantidad_total = ("cantidad", "sum"),
    promedio_cantidad = ("cantidad", "mean")
).round(2)
print(f"\n== Mostrando las estadísticas completas por estado ==\n{estadisticas_estado}") 


== Mostrando el número de pedidos por cada estado ==
estado_pedido
##                 7
###ERROR###      825
Ca               232
Cancelado      11626
De               208
Devuelto       11419
En               193
Enviado        11475
Pe               243
Pendiente      11622
dtype: int64

== Mostrando el porcentaje de pedidos por cada estado ==
estado_pedido
##              0.01
###ERROR###     1.72
Ca              0.48
Cancelado      24.30
De              0.43
Devuelto       23.86
En              0.40
Enviado        23.98
Pe              0.51
Pendiente      24.29
dtype: float64

== Mostrando el total de ventas por estado ==
estado_pedido
##                22051.05
###ERROR###     4995313.59
Ca              1701845.88
Cancelado      80947972.80
De              1274900.96
Devuelto       77001419.03
En              1157541.09
Enviado        77631408.09
Pe              1428716.12
Pendiente      83010609.32
Name: total, dtype: float64

== Mostrando el valor promedio de pedidos por estado

## Apartado 8: Método de pago

In [33]:
# 8.1 Contar el número de pedidos por método de pago
num_pedidos_pago = df_global.groupby('metodo_pago')['cantidad'].sum()
print(f"\n== Mostrando el número de pedidos por método de pago ==\n{num_pedidos_pago}") 

# 8.2 Calcular el porcentaje de uso de cada método de pago
# Echo mano de la variable asignada al dataframe con el número de pedidos por método de pago para llevar a cabo sus porcentajes
porcentajes_pagos = ((num_pedidos_pago / num_pedidos_pago.sum()) * 100).round(2)
print(f"\n== Mostrando el porcentaje de uso de cada método de pago ==\n{porcentajes_pagos}") 

# 8.3 Calcular el valor total de ventas por método de pago
total_ventas_pago = df_global.groupby('metodo_pago')['total'].sum()
print(f"\n== Mostrando el total de ventas por método de pago ==\n{total_ventas_pago}")    # Como se convirtió la columna 'total' a float, el resultado son datos totales numéricos

# 8.4 Calcular el valor promedio de pedidos por método de pago
valor_promedio_pago = (df_global.groupby('metodo_pago')['total'].mean()).round(2)
print(f"\n== Mostrando el valor promedio de pedidos por método de pago ==\n{valor_promedio_pago}") 

# 8.5 Calcular la cantidad promedio de productos por método de pago
cantidad_promedio_pago = df_global.groupby('metodo_pago')['cantidad'].mean()
print(f"\n== Mostrando la cantidad promedio de productos por método de pago ==\n{cantidad_promedio_pago}") 

# 8.6 Mostrar estadísticas completas por método de pago (numero de pedidos, total de ventas, promedio de pedidos, cantidad total, promedio de la cantidad)
estadisticas_pago = df_global.groupby("metodo_pago").agg(
    numero_pedidos = ("Id Pedido", "count"),
    total_ventas = ("total", "sum"),
    promedio_ventas = ("total", "mean"),
    cantidad_total = ("cantidad", "sum"),
    promedio_cantidad = ("cantidad", "mean")
).round(2)
print("\n== Mostrando estadísticas completas por método de pago ==")
print(estadisticas_pago)


== Mostrando el número de pedidos por método de pago ==
metodo_pago
##                    270
###ERROR###        845122
Cr                  11180
Criptomoneda     19821048
Pa                 309349
PayPal           19802984
Ta                 210393
Tarjeta          23203000
Tr                1608728
Transferencia    18798737
Name: cantidad, dtype: int64

== Mostrando el porcentaje de uso de cada método de pago ==
metodo_pago
##                0.00
###ERROR###       1.00
Cr                0.01
Criptomoneda     23.43
Pa                0.37
PayPal           23.40
Ta                0.25
Tarjeta          27.42
Tr                1.90
Transferencia    22.22
Name: cantidad, dtype: float64

== Mostrando el total de ventas por método de pago ==
metodo_pago
##                  27252.42
###ERROR###       6387671.36
Cr                1488842.62
Criptomoneda     78626106.12
Pa                2284394.84
PayPal           78313650.21
Ta                1322464.62
Tarjeta          80097439.21
Tr       

## Apartado 9: País de envío

In [48]:
# 9.1 Contar el número total de países únicos
num_paises_unicos = df_global["País envío"].nunique()
print(f"\n== Mostrando el número total de países únicos ==\n{num_paises_unicos}") 

# 9.2 Mostrar los top 10 países por número de pedidos
top_paises_pedidos = df_global.groupby("País envío")["Id Pedido"].nunique().sort_values(ascending=False).head(10)
print("\n== Mostrando los top 10 países por número de pedidos ==")
print(top_paises_pedidos)

# 9.3 Mostrar los top 10 países por valor total de ventas
top_paises_ventas = df_global.groupby("País envío")["total"].nunique().sort_values(ascending=False).head(10)
print("\n== Mostrando los top 10 países por valor total de ventas ==")
print(top_paises_ventas)

# 9.4 Calcular el valor promedio de pedidos por país (top 10)
promedio_pedidos_pais = (df_global.groupby("País envío")['total'].mean()).round(2)
print(f"\n== Mostrando el valor promedio de pedidos por país (top 10) ==\n{promedio_pedidos_pais.head(10)}")

# 9.5 Calcular la cantidad total de productos enviados por país (top 10)
productos_enviados_pais = (df_global[df_global["estado_pedido"] == "Enviado"].groupby("País envío")["cantidad"].sum().sort_values(ascending=False))
print(f"\n== Mostrando la cantidad total de productos enviados por país (top 10) ==\n{promedio_pedidos_pais.head(10)}")

# 9.6 Mostrar estadísticas del país con más pedidos (número de pedidos, total de ventas, promedio de pedido y total de cantidad)
# País con más pedidos únicos
pais_mas_pedidos = (df_global.groupby("País envío")["Id Pedido"].nunique().sort_values(ascending=False).idxmax())
# Estadísticas del país
estadisticas_pais_top = df_global[df_global["País envío"] == pais_mas_pedidos].agg(
    numero_pedidos = ("Id Pedido", "nunique"),
    total_ventas = ("total", "sum"),
    promedio_pedido = ("total", "mean"),
    cantidad_total = ("cantidad", "sum")
).round(2)
print(f"\n== Mostrando las estadísticas del país con más pedidos ({pais_mas_pedidos}) ==")
print(estadisticas_pais_top)


== Mostrando el número total de países únicos ==
354

== Mostrando los top 10 países por número de pedidos ==
País envío
###ERROR###                 608
Congo                       303
Korea                       296
Venezuela                   189
Iran                        185
Uruguay                     181
Cote d'Ivoire               179
Saint Barthelemy            179
United States of America    178
Eritrea                     178
Name: Id Pedido, dtype: int64

== Mostrando los top 10 países por valor total de ventas ==
País envío
###ERROR###         606
Congo               305
Korea               300
Iran                187
Venezuela           186
Cote d'Ivoire       180
Sierra Leone        179
Saint Barthelemy    179
Pitcairn Islands    178
Nigeria             178
Name: total, dtype: int64

== Mostrando el valor promedio de pedidos por país (top 10) ==
País envío
##                1784.14
###ERROR###       8653.31
Af                6289.30
Afghanistan       6415.70
Al         

## Apartado 10: Análisis demográfico

In [16]:
# 10.1 Contar el número total de clientes únicos


# 10.2 Distribución de clientes por género


# 10.3 Porcentaje de distribución por género


# 10.4 Distribución de clientes por nivel de fidelización


# 10.5 Estadísticas básicas de edad


# 10.6 Mostrar estadísticas completas de edad por género (número de clientes, edad promedio, edad mínima, edad máxima y mediana de edad)




## Apartado 11: Clientes por país

In [17]:
# 11.1 Contar el número total de países únicos donde viven los clientes


# 11.2 Mostrar los top 15 países con más clientes


# 11.3 Calcular el porcentaje de clientes por país (top 10)


# 11.4 Mostrar estadísticas (número de clientes, edad promedio, edad mínima, edad máxima y mediana de edad) de edad por país (top 5 países)


# 11.5 Mostrar distribución de género por país (top 3 países)


# 11.6 Mostrar distribución de nivel de fidelización por país (top 3 países)




## Apartado 12: Clientes por ciudad

In [18]:
# 12.1 Mostrar las top 20 ciudades con más clientes


# 12.2 Mostrar las ciudades con exactamente 1 cliente


# 12.3 Mostrar las ciudades con más de 10 clientes




## Apartado 13: Categoría de productos

In [19]:
# 13.1 Mostrar las categorías cuyo nombre tiene más de 6 caracteres y que tienen entre 50 y 160 productos


# 13.2 Mostrar las categorías que contienen la letra "o" (mayúscula o minúscula) y tienen menos de 120 productos



## Apartado 14: Precios

In [20]:
# 14.1 Mostrar estadísticas básicas de precios base (precio promedio, precio mínimo, precio máximo y mediana)


# 14.2 Calcular el precio promedio por categoría


# 14.3 Encontrar los productos más caros y más baratos


# 14.4 Mostrar los top 10 productos más caros




## Apartado 15: Productos más vendidos

In [21]:

# 15.1 Obtener el producto más vendido (cantidad vendida, total vendido, nombre del producto y categoría)



## Apartado 16: Análisis temporal


In [22]:
# 16.1 Obtener información sobre las ventas por mes (número total de ventas, número de pedidos y cantidad total)


# 16.2 Obten la misma información pero en lugar de por mes, por semana.



## Apartado 17: Duplicados

In [ ]:
# 17.1 Contar filas duplicadas en cada uno de los datasets originales.



## Apartado 18: Valores faltantes

In [24]:
# 18.1 Obtener el porcentaje de valores faltantes en cada columna del dataset.


## Apartado 19: Valoración final

Tras completar este ejercicio, ¿qué conclusiones has obtenido acerca de los datos? ¿Consideras que sería necesario aplicar algún tipo de preprocesamiento o crees que los datos son adecuados tal como están? Apoya tu valoración con ejemplos concretos de columnas que ilustren tu análisis.

<!-- Responde aquí al apartado 19 -->
**Responder**

